In [2]:
import torch
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import os

In [7]:
# 1.가상의 CSV 파일 데이터 생성 -->
csv_data = {
    'feature1':[0.1,0.2,0.3,0.4,0.5,0.6],
    'feature2':[0.6,0.5,0.4,0.3,0.2,0.1],
    'feature3':[1.0,0.9,0.8,0.7,0.6,0.5],
    'target':[1,0,1,0,1,0],
}

df = pd.DataFrame(csv_data) # dataFrame object생성
csv_path = 'dummy_data.csv'
df.to_csv(csv_path,index=False)
print(df)

   feature1  feature2  feature3  target
0       0.1       0.6       1.0       1
1       0.2       0.5       0.9       0
2       0.3       0.4       0.8       1
3       0.4       0.3       0.7       0
4       0.5       0.2       0.6       1
5       0.6       0.1       0.5       0


In [32]:
class CustomCVSDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """데이터셋 초기화 메소드
        Args:
            csv_file (str): CSV 파일경로(데이터 원천 지정)
            transform (torchvision.transform.Compose, optional): 데이터에 적용한 전처리(선택사항), None->option
        """
        # 1.파일 읽어서 데이터 프레임 생성
        self.data_frame = pd.read_csv(csv_file)
        # 2. 데이터와 레이블을 분리
        self.features = self.data_frame.iloc[:,:-1].values
        self.target = self.data_frame.iloc[:,-1].values
        # transform을 항상 설정
        self.transform = transform
    
    def __len__(self) -> int:
        return len(self.data_frame)
    
    def __getitem__(self, idx: int):
        feature_sample = self.features[idx]
        target_sample = self.target[idx]
        
        if self.transform:  # None이 아니면 전처리 적용
            feature_sample = self.transform(feature_sample)
        
        return torch.tensor(feature_sample, dtype=torch.float32), torch.tensor(target_sample, dtype=torch.long)

In [ ]:
# 목표: 파일로 만들기
class CustomCVSDataset(Dataset):
    def __init__(self,csv_file,transform=None):
        """데이터셋 초기화 메소드

        Args:
            csv_file (str): CSV 파일경로(데이터 원천 지정)
            transform (torchvision.transform.Compose, optional): 데이터에 적용한 전처리(선택사항), None->option
        """

        # 1.파일 읽어서 데이터 프레임 생성
        self.data_frame =  pd.read_csv(csv_file) # data frame 생성
        # 2. 데이터와 레이블을 분리
        self.features = self.data_frame.iloc[:,:-1].values # 결국 iloc 쓸려고 pandas 로 바꾼거임
        self.target = self.data_frame.iloc[:,-1].values

        self.transform = transform


    def __len__(self) -> int: #크기 반환
        return len(self.data_frame)

    def __getitem__(self,idx:int):
        feature_sample = self.features[idx]
        target_sample = self.target[idx]


        if self.transform: # None(False) 아니면,전처리기 존재하면
            feature_sample = self.transform(feature_sample) 

        return torch.tensor(feature_sample,dtype=torch.float32),torch.tensor(target_sample,dtype=torch.long)
        # long 주는 이유: 문제가 다중분류 문제다, 다중분류에 사용될 dataset이다.
        # 원래답(Target)은 손실함수에 들어감, CrossEntropy(다중분류에 사용됨)에 들어감
        # 다중분류에 사용되는 손실함수(CrossEntrophyLoss) int64  [int32는 에러뜸]
        # int64 = long
        # 결과(label)의 dataset까지 생각해야하니까 조심해ㅑ함

## 근데 이거...old 한 방식이다.

In [33]:
# Google + type hint
# [참고] mkdocs build ---> 웹페이지로 만들어 놓음
def exam_func(items:list[str],truncate:bool=False) -> int: # 복수형일때는 list[str]사용
    """summary: 함수의 용도 명시
        리스트 내의 문자열 처리,횟수 반환

    Args:
        items (_type_): _description_
        truncate (_type_): _description_

    Raises : 
        valueError: 에러발생 원인 

    Exampels :
    """

    return 1

## 최신방식: doctring, 타입힌트 같이 쓰임
# 이렇게 독스트링을 만들어 놓는거를 습관화 해야함 --> 받는 사람이 좋아함

In [34]:
# 3. Dataset생성
custom_dataset= CustomCVSDataset(csv_path) # transform 안주면 None 으로 들어감

# 4. DataLoader 생성
custom_dataloader = DataLoader(custom_dataset,batch_size=2,shuffle=True)

In [ ]:
# 데이터 로더에서 한 배치를 가져와보기
feature_batch, target_batch = next(iter(custom_dataloader))
print(f'Features batch shape : {feature_batch.shape}')
print(f'Features batch : {feature_batch}')
print(f'Targets batch shape:{target_batch.shape}')
print(f'Targets batch: {target_batch}')

# 실행할때마다 결과다랄라짐

Features batch shape : torch.Size([2, 3])
Features batch : tensor([[0.4000, 0.3000, 0.7000],
        [0.1000, 0.6000, 1.0000]])
Targets batch shape:torch.Size([2])
Targets batch: tensor([0, 1])


In [ ]:
# 파일 정리
# os : 운영체제 관련 모듈
# 컨테이너 -> dumyy 파일 삭제됨
os.remove(csv_path) 